In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from rapidfuzz import process, fuzz
import re
import time

df = pd.read_csv('result.csv')
df_filtered = df[df['minutes'] > 900].copy()

PATH = "c:\\Users\\ADMIN\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"
service = Service(executable_path=PATH)
driver = webdriver.Chrome(service=service)

all_data = []
for page in range(1, 23):
    if page == 1:
        url = 'https://www.footballtransfers.com/us/values/players/most-valuable-soccer-players/playing-in-uk-premier-league'
    else:
        url = f'https://www.footballtransfers.com/us/values/players/most-valuable-soccer-players/playing-in-uk-premier-league/{page}'
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('tbody', {'id': 'player-table-body'})

    rows = table.find_all('tr')
    print(f"✅ Trang {page}: {len(rows)} cầu thủ")

    for row in rows:
        cols = row.find_all('td')
        data = []
        for idx, col in enumerate(cols):
            if idx == 2:  # Cột "Player"
                span = col.find('span')
                name = span.get_text(strip=True) if span else col.get_text(strip=True)
                data.append(name)
            else:
                data.append(col.get_text(strip=True))
        if data:
            all_data.append(data)

driver.quit()

df_values = pd.DataFrame(all_data)
df_values.columns = ['Skill', 'Rank', 'Player', 'Age', 'Club', 'MarketValue']

def clean_player_name(name):
    name = str(name)

    # Tên bị lặp (ví dụ: CaicedoCaicedo)
    match = re.match(r'^([A-Za-zÀ-ÿ\.\- ]+?)\1', name)
    if match:
        return match.group(1).strip()

    # Lấy 2 từ đầu tiên
    words = name.split()
    if len(words) >= 2:
        return f"{words[0]} {words[1]}"
    return name

df_values['CleanedPlayer'] = df_values['Player'].apply(clean_player_name)
# df_values.to_csv('df_values.csv', index = False, encoding= 'utf - 8 - sig')
df_filtered['CleanedPlayer'] = df_filtered['player'].apply(clean_player_name)

web_players = df_values['CleanedPlayer'].tolist()
value_dict = dict(zip(df_values['CleanedPlayer'], df_values['MarketValue']))

def get_market_value_fuzzy(player_name):
    match, score, _ = process.extractOne(player_name, web_players, scorer=fuzz.token_sort_ratio)
    if score >=85:  
        return value_dict[match]
    return 'N/A'

df_filtered['MarketValue'] = df_filtered['CleanedPlayer'].apply(get_market_value_fuzzy)


final_df = df_filtered[['player', 'minutes', 'MarketValue']].copy()
final_df.insert(0, 'stt', range(1, len(final_df) + 1))

final_df.to_csv('b4.csv', index=False, encoding='utf-8-sig')
print("\n✅ Đã lưu file b4.csv thành công!")
print(final_df.head(10))

✅ Trang 1: 25 cầu thủ
✅ Trang 2: 25 cầu thủ
✅ Trang 3: 25 cầu thủ
✅ Trang 4: 25 cầu thủ
✅ Trang 5: 25 cầu thủ
✅ Trang 6: 25 cầu thủ
✅ Trang 7: 25 cầu thủ
✅ Trang 8: 25 cầu thủ
✅ Trang 9: 25 cầu thủ
✅ Trang 10: 25 cầu thủ
✅ Trang 11: 25 cầu thủ
✅ Trang 12: 25 cầu thủ
✅ Trang 13: 25 cầu thủ
✅ Trang 14: 25 cầu thủ
✅ Trang 15: 25 cầu thủ
✅ Trang 16: 25 cầu thủ
✅ Trang 17: 25 cầu thủ
✅ Trang 18: 25 cầu thủ
✅ Trang 19: 25 cầu thủ
✅ Trang 20: 25 cầu thủ
✅ Trang 21: 25 cầu thủ
✅ Trang 22: 6 cầu thủ

✅ Đã lưu file b4.csv thành công!
    stt              player  minutes MarketValue
1     1      Aaron Ramsdale     2160      €18.7M
2     2   Aaron Wan-Bissaka     2704      €26.9M
3     3  Abdoulaye Doucouré     2245       €5.8M
6     4      Adam Armstrong     1248         N/A
8     5          Adam Smith     1146       €1.5M
10    6        Adam Wharton     1088      €48.9M
11    7        Adama Traoré     1482         €8M
13    8  Alejandro Garnacho     1876      €60.7M
14    9          Alex Iwobi  